In [1]:
import os 
os.chdir("..")

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import json

from utils.visualization import plot_molecules, plot_molecule
from utils.preprocessing import get_average_trajectory_positions, get_time_distance_matrix
import processor.data as data_processor
import processor.graph as graph_processor
import utils.metrics as metrics
from utils.export import *

In [3]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available. Using:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU not available. Using CPU.")

GPU not available. Using CPU.


# Load data

In [4]:
integrin = "aVb3"    # "a5b1"
data_type = "clamp"  # Ramp 
u, extensions, config = data_processor.load_data(data_type, integrin)

In [5]:
# visualize 
t = 0
rep = 0

# set to first frame 
u[rep][extensions[t]].trajectory[0] 

plot_molecules(
    [
        u[rep][extensions[t]].select_atoms(f"protein and name CA").positions
    ]
)

# Define domains

In [6]:
# define domains 
ext = extensions[0]

if integrin == "aVb3":
    domain_to_residues = {
        "beta-propeller": u[0][ext].select_atoms(f"protein and name CA and segid A and resid 1-438").atoms.resindices,
        "thigh": u[0][ext].select_atoms(f"protein and name CA and segid A and resid 439-592").atoms.resindices,
        "loopA": u[0][ext].select_atoms(f"protein and name CA and segid A and resid 593-601").atoms.resindices,
        "calf1": u[0][ext].select_atoms(f"protein and name CA and segid A and resid 602-738").atoms.resindices,
        "calf2": u[0][ext].select_atoms(f"protein and name CA and segid A and resid 739-967").atoms.resindices,
        "transmembrane-alpha": u[0][ext].select_atoms(f"name CA and segid A and resid 968-984").atoms.resindices,
        "cytoplasmic-alpha": u[0][ext].select_atoms(f"name CA and segid A and resid 985-1016").atoms.resindices,
        "psi": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 1-56").atoms.resindices,
        "hybrid": u[0][ext].select_atoms(f"protein and name CA and segid B and (resid 57-108 or resid 353-433)").atoms.resindices,
        "betaI": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 109-352").atoms.resindices,
        "loopB": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 434-436").atoms.resindices,
        "egf1": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 437-472").atoms.resindices,
        "egf2": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 473-522").atoms.resindices,
        "egf3": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 523-559").atoms.resindices,
        "egf4": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 560-600").atoms.resindices,
        "betaTD": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 601-695").atoms.resindices,
        "transmembrane-beta": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 696-725").atoms.resindices,
        "cytoplasmic-beta": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 726-763").atoms.resindices,
    }
    
elif integrin == "alphaVbeta3":
    domain_to_residues = {
        "beta-propeller": u[0][ext].select_atoms(f"protein and name CA and segid A and resid 1-438").atoms.resindices,
        "thigh": u[0][ext].select_atoms(f"protein and name CA and segid A and resid 439-592").atoms.resindices,
        "loopA": u[0][ext].select_atoms(f"protein and name CA and segid A and resid 593-601").atoms.resindices,
        "calf1": u[0][ext].select_atoms(f"protein and name CA and segid A and resid 602-738").atoms.resindices,
        "calf2": u[0][ext].select_atoms(f"protein and name CA and segid A and resid 739-956").atoms.resindices,
        "transmembrane-alpha": u[0][ext].select_atoms(f"name CA and segid A and resid 957-984").atoms.resindices,
        # "calf2": u[0][ext].select_atoms(f"protein and name CA and segid A and resid 739-967").atoms.resindices,
        # "transmembrane-alpha": u[0][ext].select_atoms(f"name CA and segid A and resid 968-984").atoms.resindices,
        "cytoplasmic-alpha": u[0][ext].select_atoms(f"name CA and segid A and resid 985-1016").atoms.resindices,
        "psi": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 1-56").atoms.resindices,
        "hybrid": u[0][ext].select_atoms(f"protein and name CA and segid B and (resid 57-108 or resid 353-433)").atoms.resindices,
        "betaI": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 109-352").atoms.resindices,
        "loopB": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 434-436").atoms.resindices,
        "egf1": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 437-472").atoms.resindices,
        "egf2": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 473-522").atoms.resindices,
        "egf3": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 523-559").atoms.resindices,
        "egf4": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 560-600").atoms.resindices,
        "betaTD": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 601-692").atoms.resindices,
        "transmembrane-beta": u[0][ext].select_atoms(f"name CA and segid B and resid 693-725").atoms.resindices,
        # "betaTD": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 601-695").atoms.resindices,
        # "transmembrane-beta": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 696-725").atoms.resindices,
        "cytoplasmic-beta": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 726-763").atoms.resindices,
    }
    
elif integrin == "a5b1":
    domain_to_residues = {
        "beta-propeller": u[0][ext].select_atoms(f"protein and name CA and segid A and resid 1-449").atoms.resindices,
        "thigh": u[0][ext].select_atoms(f"protein and name CA and segid A and resid 450-602").atoms.resindices,
        "loopA": u[0][ext].select_atoms(f"protein and name CA and segid A and resid 603-611").atoms.resindices,
        "calf1": u[0][ext].select_atoms(f"protein and name CA and segid A and resid 612-748").atoms.resindices,
        "calf2": u[0][ext].select_atoms(f"protein and name CA and segid A and resid 749-947").atoms.resindices,
        "betaI": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 121-360").atoms.resindices,
        "hybrid": u[0][ext].select_atoms(f"protein and name CA and segid B and (resid 65-120 or resid 361-441)").atoms.resindices,
        "psi": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 1-64").atoms.resindices,
        "loopB": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 442-444").atoms.resindices,
        "egf1": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 445-480").atoms.resindices,
        "egf2": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 481-533").atoms.resindices,
        "egf3": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 534-570").atoms.resindices,
        "egf4": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 571-610").atoms.resindices,
        "betaTD": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 611-703").atoms.resindices
    }
    
elif integrin == "alpha2bbeta3":
    # https://www.cell.com/molecular-cell/fulltext/S1097-2765(08)00839-3?_returnURL=https%3A%2F%2Flinkinghub.elsevier.com%2Fretrieve%2Fpii%2FS1097276508008393%3Fshowall%3Dtrue 
    # Zhu, Jianghai, et al. "Structure of a complete integrin ectodomain in a physiologic resting state and activation and deactivation by applied forces." Molecular cell 32.6 (2008): 849-861.
    domain_to_residues = {
        "beta-propeller": u[0][ext].select_atoms(f"protein and name CA and segid A and resid 1-451").atoms.resindices,
        "thigh": u[0][ext].select_atoms(f"protein and name CA and segid A and resid 452-601").atoms.resindices,
        "calf1": u[0][ext].select_atoms(f"protein and name CA and segid A and resid 602-743").atoms.resindices,
        "calf2": u[0][ext].select_atoms(f"protein and name CA and segid A and resid 744-965").atoms.resindices,
        "transmembrane-alpha": u[0][ext].select_atoms(f"name CA and segid A and resid 966-1080").atoms.resindices,
        "cytoplasmic-alpha": u[0][ext].select_atoms(f"name CA and segid A and resid 989-1080").atoms.resindices,
        "psi": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 1008-1063").atoms.resindices,
        "hybrid": u[0][ext].select_atoms(f"protein and name CA and segid B and (resid 1064-1115 or resid 1360-1440)").atoms.resindices,
        "betaI": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 1116-1359").atoms.resindices,
        "egf1": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 1444-1479").atoms.resindices,
        "egf2": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 1480-1529").atoms.resindices,
        "egf3": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 1530-1566").atoms.resindices,
        "egf4": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 1567-1607").atoms.resindices,
        "betaTD": u[0][ext].select_atoms(f"protein and name CA and segid B and resid 1608-1699").atoms.resindices,
        "transmembrane-beta": u[0][ext].select_atoms(f"name CA and segid B and resid 1700-1732").atoms.resindices,
        "cytoplasmic-beta": u[0][ext].select_atoms(f"name CA and segid B and resid 1733-1770").atoms.resindices,
    }
else:
    domain_to_residues = {
        "molecule": u[0][ext].select_atoms(f"protein and name CA").atoms.resindices,
    }
    
residue_to_domain = {}
for domain, residues in domain_to_residues.items():
    for residue in residues:
        residue_to_domain[residue] = domain

domain_to_chain = {
    'beta-propeller': "A", 
    'thigh': "A", 
    'loopA': "A", 
    'calf1': "A", 
    'calf2': "A", 
    'transmembrane-alpha': "A", 
    'cytoplasmic-alpha': "A", 
    'psi': "B", 
    'hybrid': "B",
    'betaI': "B", 
    'loopB': "B", 
    'egf1': "B", 
    'egf2': "B", 
    'egf3': "B", 
    'egf4': "B", 
    'betaTD': "B", 
    'transmembrane-beta': "B", 
    'cytoplasmic-beta': "B"
}


# Construct graph

In [ ]:
# construct molecular graph sequence 
graph_sequences, dygraph_sequences, resindices_to_index, dist_matrices = graph_processor.contruct_graph_dygraph(
    u=u,
    extensions=extensions,
    config=config,
    residue_to_domain=residue_to_domain,
    warm_up_frames=1,
    node_attributes="coords",
    bound_thd = 5, 
    pval_thd = 1e-5
)

ext: 3nm
There are 1623 residues and 801 timestamps


100%|██████████| 800/800 [00:00<00:00, 1057.81it/s]


There are 1623 residues and 801 timestamps
ext: 11nm
There are 1623 residues and 801 timestamps


100%|██████████| 800/800 [00:00<00:00, 1543.77it/s]


There are 1623 residues and 801 timestamps
ext: 16nm
There are 1623 residues and 801 timestamps


...

There are 1623 residues and 801 timestamps
ext: 16nm
There are 1623 residues and 801 timestamps


100%|██████████| 800/800 [00:00<00:00, 1565.80it/s]


There are 1623 residues and 801 timestamps
ext: 18nm
There are 1623 residues and 801 timestamps


100%|██████████| 800/800 [00:00<00:00, 1320.47it/s]


There are 1623 residues and 801 timestamps


# Run model

In [ ]:
#import helper libraries
from dynamicgem.utils      import graph_util, plot_util, dataprep_util
from dynamicgem.evaluation import visualize_embedding as viz
from dynamicgem.visualization import plot_dynamic_sbm_embedding
from dynamicgem.evaluation import evaluate_graph_reconstruction as gr
from dynamicgem.graph_generation import dynamic_SBM_graph as sbm

#import the methods
from dynamicgem.embedding.dynSDNE import DynSDNE
from dynamicgem.embedding.sdne_dynamic    import SDNE
from dynamicgem.embedding.ae_static    import AE
from dynamicgem.embedding.dynAE        import DynAE
from dynamicgem.embedding.dynRNN       import DynRNN
from dynamicgem.embedding.dynAERNN     import DynAERNN
from scipy import sparse

model_type = "dynamicgem"

In [ ]:
from utils.clustering import get_communities_with_optimal_clusters, find_optimal_clusters_per_timepoint

In [ ]:
dim_emb = 128
lookback = 1
n_epochs = 500 # 400 
length = len(extensions)
rep = 0
graph_list = graph_sequences[rep]

In [ ]:
# # AE Static
# embedding = AE(d            = dim_emb, 
#                  beta       = 5, 
#                  nu1        = 1e-6, 
#                  nu2        = 1e-6,
#                  K          = 3, 
#                  n_units    = [500, 300, ],
#                  n_iter     = n_epochs, 
#                  xeta       = 1e-4,
#                  n_batch    = 100,
#                  modelfile  = ['./intermediate/enc_modelsbm.json',
#                              './intermediate/dec_modelsbm.json'],
#                  weightfile = ['./intermediate/enc_weightssbm.weights.h5',
#                              './intermediate/dec_weightssbm.weights.h5'])
# embs  = []
# t1 = time()
# # ae static
# for t in range(length):
#     emb, _= embedding.learn_embeddings(graph_list[t])
#     embs.append(emb)
# print(embedding._method_name+':\n\tTraining time: %f' % (time() - t1))

# # viz.plot_static_sbm_embedding(embs[-4:], dynamic_sbm_series[-4:])   

In [ ]:
# #dynGEM
# # embedding = DynSDNE(d           = dim_emb,
# embedding = SDNE(d           = dim_emb,
#                  beta           = 5,
#                  alpha          = 0.0,
#                  nu1            = 1e-6,
#                  nu2            = 1e-6,
#                  rho            = 0.3, 
#                  K              = 3,
#                  n_iter_subs    = 5, 
#                  n_units        = [500, 300,],
#                  n_iter         = 10,
#                  xeta           = 0.01,
#                  n_batch        = 100,
#                  modelfile      = ['./intermediate/enc_model_dynSDNE.json', 
#                                    './intermediate/dec_model_dynSDNE.json'],
#                  weightfile     = ['./intermediate/enc_weights_dynSDNE.weights.h5', 
#                                    './intermediate/dec_weights_dynSDNE.weights.h5']
#                  )
# embs = []
# t1 = time()
# for temp_var in range(lookback+1, length+1):
#     print(temp_var)
#     emb = embedding.learn_embeddings(graphs[:temp_var])
#     print(emb.shape)
#     embs.append(emb)
# print (embedding._method_name+':\n\tTraining time: %f' % (time() - t1))
# plt.figure()
# plt.clf()    
# plot_dynamic_sbm_embedding.plot_dynamic_sbm_embedding_v2(embs[-5:-1], dynamic_sbm_series[-5:])    
# plt.show()

In [ ]:
graph_list = [graph_list[0]] + graph_list
len(graph_list)

In [ ]:
#dynAERNN
embedding = DynAERNN(d   = dim_emb,
            beta           = 5,
            n_prev_graphs  = lookback,
            nu1            = 1e-6,
            nu2            = 1e-6,
            n_aeunits      = [300, dim_emb],
            n_lstmunits    = [300, dim_emb],
            rho            = 0.3,
            n_iter         = 20, # 50 same results
            xeta           = 0.001,
            n_batch        = 500,
            modelfile      = ['./intermediate/enc_model_dynAERNN.json', 
                              './intermediate/dec_model_dynAERNN.json'],
            weightfile     = ['./intermediate/enc_weights_dynAERNN.weights.h5', 
                              './intermediate/dec_weights_dynAERNN.weights.h5'],
            )

embs = []
t1 = time()
for t in range(lookback+1, length+1+1):
    emb, _ = embedding.learn_embeddings(graph_list[:t])
    embs.append(emb)
print (embedding._method_name+':\n\tTraining time: %f' % (time() - t1))
# plt.figure()
# plt.clf()    
# plot_dynamic_sbm_embedding.plot_dynamic_sbm_embedding_v2(embs[-5:-1], dynamic_sbm_series[-5:])    
# plt.show()

/opt/homebrew/Caskroom/miniconda/base/envs/research/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 472ms/step - loss: 62.8878
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 926ms/step - loss: 62.7846
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 964ms/step - loss: 61.0560
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 925ms/step - loss: 61.0392
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 905ms/step - loss: 60.6828
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 951ms/step - loss: 60.3862
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 941ms/step - loss: 60.5866
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 935ms/step - loss: 59.9330
Epoch 9/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 925ms/step - loss: 59.3406
Epoch 10/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 921ms/step - loss: 58.7870
Epoch 11/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 931ms/step - loss: 58.2913
Epoch 12/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 929ms/step - loss: 56.8330
Epoch 13/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 931ms/step - loss: 55.3122
Epoch 14/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 916ms/step - loss: 53.5795
Epoch 15/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 924ms/step - loss: 51.8448
Epoc

In [ ]:
adjacency_list = []
for t in range(len(extensions)):
    adjacency_list.append(sparse.csr_matrix(nx.to_scipy_sparse_array(graph_sequences[rep][t], format='csr')))
    
### Clustering 
# Use this to find optimal number of clusters
# optimal_n_clusters, all_silhouette_scores = find_optimal_clusters_per_timepoint(embs, plot=False, max_clusters=40)
communities = get_communities_with_optimal_clusters(embs, [30,30,30,30], residue_indices, adjacency_list)

t0: Modularity=0.1526, Conductance=0.8042
t1: Modularity=0.1147, Conductance=0.8410
t2: Modularity=0.0955, Conductance=0.8636
t3: Modularity=0.1059, Conductance=0.8473


In [ ]:
#dynAERNN
dim_emb = 32
embedding = DynAERNN(d   = dim_emb,
            beta           = 5,
            n_prev_graphs  = lookback,
            nu1            = 1e-6,
            nu2            = 1e-6,
            n_aeunits      = [64, 32], # [500, 300],
            n_lstmunits    = [64, 32],# [500, dim_emb],
            rho            = 0.3,
            n_iter         = 50,
            xeta           = 0.01,
            n_batch        = 500,
            modelfile      = ['./intermediate/enc_model_dynAERNN.json', 
                              './intermediate/dec_model_dynAERNN.json'],
            weightfile     = ['./intermediate/enc_weights_dynAERNN.weights.h5', 
                              './intermediate/dec_weights_dynAERNN.weights.h5'],
            )

embs = []
t1 = time()
for t in range(lookback+1, length+1+1):
    emb, _ = embedding.learn_embeddings(graph_list[:t])
    embs.append(emb)
print (embedding._method_name+':\n\tTraining time: %f' % (time() - t1))
# plt.figure()
# plt.clf()    
# plot_dynamic_sbm_embedding.plot_dynamic_sbm_embedding_v2(embs[-5:-1], dynamic_sbm_series[-5:])    
# plt.show()

Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 535ms/step - loss: 63.0304
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - loss: 60.8544
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - loss: 59.6787
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - loss: 56.9932 
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - loss: 55.2014
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - loss: 53.5814
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - loss: 51.9386 
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - loss: 50.0829  
Epoch 9/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 987ms/step - loss: 48.0260
Epoch 10/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 965ms/step - loss: 46.3747
Epoch 11/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 937ms/step - loss: 45.1691
Epoch 12/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 930ms/step - loss: 43.7678
Epoch 13/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 938ms/step - loss: 42.2113
Epoch 14/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 977ms/step - loss: 40.7503
Epoch 15/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 924ms/step - loss: 39.3430
Epoch 16/50
3/3 ━━━━━

In [ ]:
adjacency_list = []
for t in range(len(extensions)):
    adjacency_list.append(sparse.csr_matrix(nx.to_scipy_sparse_array(graph_sequences[rep][t], format='csr')))
    
### Clustering 
# Use this to find optimal number of clusters
# optimal_n_clusters, all_silhouette_scores = find_optimal_clusters_per_timepoint(embs, plot=False, max_clusters=40)
communities = get_communities_with_optimal_clusters(embs, [30,30,30,30], residue_indices, adjacency_list)

t0: Modularity=0.1889, Conductance=0.7764
t1: Modularity=0.1929, Conductance=0.7725
t2: Modularity=0.1902, Conductance=0.7746
t3: Modularity=0.1324, Conductance=0.8327
